# Importing Packages

In [73]:
using JuMP
using GLPK

### Storage System Constraints

Constraints:                                     Shadow Variables:

0 <= e[t]     (for all T in T)                 : (v_low[t])
e[t] <= maxS  (for all T in T)                 : v_up[t]
e[t] = e[t-1]+b[t] (for all t in T \ {1})      : (rho[t])
e[1] = E_init + b[1]                           : (rho[1])
e[T] = E_end                                   : (xi)

Stuff that can be left out based on Frolke chapter 5:
no networks constraints, therefore we can simply model as a single energy storage system
Discharging and charging losses aren't modelled, therefore a single variable suffices (b, which is positive when charging, negative discharging)

## FIRST DRAFT OF SPLIT-HORIZON MARKET CLEARING MODEL

### Load Packages

In [74]:
#import Pkg; Pkg.add("CSV")
#import Pkg; Pkg.add("DataFrames")
#cd("C:\\Users\\jonas\\Documents\\Jonas DTU\\MSc Thesis\\Modelling")

using JuMP
using HiGHS
using CSV
using DataFrames

### Load data from excel files and transform into matrix form


In [75]:
gen_df = CSV.read("data_gen.csv", DataFrame)
load_df = CSV.read("data_load.csv", DataFrame)

#Creating Maximum Generation matrix
P =  Array{Float64}(undef, maximum(gen_df.gen), maximum(gen_df.time))
C =  Array{Float64}(undef, maximum(gen_df.gen), maximum(gen_df.time))
for i in 1:nrow(gen_df)
    P[gen_df.gen[i],gen_df.time[i]] = gen_df.max[i]
    C[gen_df.gen[i],gen_df.time[i]] = gen_df.cost[i]
end
# Now we can acces parameters of generators as: max output = P[g,t],   Cost = C[g,t]

#Creating Maximum Consumption matrix
D = Array{Float64}(undef, maximum(load_df.load), maximum(load_df.time))
U = Array{Float64}(undef, maximum(load_df.load), maximum(load_df.time))
for i in 1:nrow(load_df)
    D[load_df.load[i],load_df.time[i]] = load_df.max[i]
    U[load_df.load[i],load_df.time[i]] = load_df.utility[i]
end
# Now we can acces parameters of loads as: max consumption = D[g,t],   Utility = U[g,t]




## The Full Horizon Problem

In [77]:
#Number of days
days = 2
#Hours per day
hours = 1
#Total time
time = days*hours

# Sets
T = 1:size(D)[2]
L = 1:size(D)[1]
G = 1:size(P)[1]

# Storage Parameters
S = 2.5; 
E_init = 0;

#Model 😺
model = Model(HiGHS.Optimizer)
set_silent(model)

### Variables
## Variables Market Clearing
@variable(model, b[T] )            # Energy Charged or discharged for t
@variable(model, d[L, T] >= 0)   # Demand of load l at t
@variable(model, p[G, T] >= 0)   # Production of g at t
@variable(model, e[T] >= 0)        # State of Energy at the end of T

### Constraints 
## Storage Constraints (1a-1d)
@constraint(model, Stor1[t in T],   0 <= e[t] <= S)          #1a (v_low[t] , v_up[t])
@constraint(model, Stor2[t in T;t!=1],   e[t] == e[t-1]+b[t])     #1b (rho[t])
@constraint(model, Stor3, e[1] == E_init+b[1])                 #1c (rho[1])
#@constraint(model  e[H] == E_end)                      #1d (xi)  # Delete this constraint for the full horizon problem

####### Market Clearing Formulation
@objective(model, Max, sum( sum(U[l,t]*d[l,t] for l in L) -  sum(C[g,t]*p[g,t] for g in G) for t in T) )
### Subject to
@constraint(model, Balance[t in T], sum(d[l,t] for l in L) + b[t] - sum(p[g,t] for g in G) == 0 )  # 2a
@constraint(model, Gen[g in G, t in T], 0 <= p[g,t] <= P[g,t])
@constraint(model, Load[l in L, t in T], 0 <= d[l,t] <= D[l,t])


#************************************************************************
# Solve
solution = optimize!(model)
println("Termination status: $(termination_status(model))")
#************************************************************************

if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(model))")
    println("Solution:")
    for t in T
        println("The Market price in t=", t," is λ=" , dual(Balance[t]))
        println("Storage level at end of t=", t, " is e=", value(e[t]))
    end
else
    println("No optimal solution available")
end




Termination status: OPTIMAL
Optimal objective value: 27.0
Solution:
The Market price in t=1 is λ=-5.0
Storage level at end of t=1 is e=1.0
The Market price in t=2 is λ=-5.0
Storage level at end of t=2 is e=0.0


In [71]:
dual(Balance[1])

-5.0

## The Split-Horizon Problem (future-aware-plus)
##### In this problem we use the solution attained in the full-horizon problem in our objective function etc.

In [78]:
# Sets
T = 1:2
L = 1:1
G = 1:2
#end of time horizon 1 
H = 1
# Storage Parameters
S = 2.5; 
E_init = 0;
E_end = 1;

#Model 😺
model = Model(HiGHS.Optimizer)

### Variables
## Variables Market Clearing
@variable(model, b[T] )            # Energy Charged or discharged for t
@variable(model, d[L, T] >= 0)   # Demand of load l at t
@variable(model, p[G, T] >= 0)   # Production of g at t
@variable(model, e[T] >= 0)        # State of Energy at the end of T

### Constraints 
## Storage Constraints (1a-1d)
@constraint(model, Stor1_low[t in T], 0 <= e[t] )             #1a (v_low[t])
@constraint(model, Stor1_up[t in T],   e[t] <= S)             #1a (v_up[t])
@constraint(model, Stor2[t in T;t!=1],   e[t] == e[t-1]+b[t])   #1b (rho[t])
@constraint(model, Stor3, e[1] == E_init+b[1])                  #1c (rho[1])
@constraint(model  Stor4, e[H] == E_end)                        #1d (xi)  

####### Market Clearing Formulation
@objective(model, Max, sum( sum(U[l,t]*d[l,t] for l in L) -  sum(C[g,t]*p[g,t] for g in G) for t in T) + e[H]*dual(rho[H])) # 2d 
### Subject to
@constraint(model, Balance[t in T], sum(d[l,t] for l in L) + b[t] - sum(p[g,t] for g in G) == 0 )  # 2b (λ[t])
@constraint(model, Gen[g in G, t in T], 0 <= p[g,t] <= P[g,t])                                     # 2c (μ_low[g,t], μ_up[g,t])
@constraint(model, Load[l in L, t in T], 0 <= d[l,t] <= D[l,t])                                    # 2d (X_low[g,t], X_up[g,t])


#************************************************************************
# Solve
solution = optimize!(model)
println("Termination status: $(termination_status(model))")
#************************************************************************
if termination_status(model) == MOI.OPTIMAL
    println("Optimal objective value: $(objective_value(model))")
    println("Solution:")
    for t in T
        println("The Market price in t=", t," is λ=" , dual(Balance[t]))
        println("Storage level at end of t=", t, " is e=", value(e[t]))
    end
else
    println("No optimal solution available")
end





ErrorException: syntax: missing comma or ) in argument list